In [1]:
import logging
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

logging.basicConfig(level=logging.INFO)

class BrowserHandler:
    chrome_driver = None

    @staticmethod
    def initialize_browser():
        """Initialize WebDriver and open the website."""
        BrowserHandler.chrome_driver = webdriver.Chrome()
        BrowserHandler.chrome_driver.maximize_window()
        BrowserHandler.chrome_driver.get("https://app-staging.nokodr.com/")

    @staticmethod
    def shutdown_browser():
        """Close the browser."""
        if BrowserHandler.chrome_driver:
            BrowserHandler.chrome_driver.quit()

class ResetPasswordTests(BrowserHandler):

    @staticmethod
    def navigate_to_reset():
        """Navigate to the forgot password page."""
        BrowserHandler.chrome_driver.get("https://app-staging.nokodr.com/")
        wait = WebDriverWait(BrowserHandler.chrome_driver, 10)
        forgot_link = (By.LINK_TEXT, "Forgot Password?")
        wait.until(EC.element_to_be_clickable(forgot_link)).click()
        wait.until(EC.presence_of_element_located((By.ID, "email")))

    @staticmethod
    def test_empty_email():
        logging.info("Testing empty email validation...")
        wait = WebDriverWait(BrowserHandler.chrome_driver, 10)
        proceed_button = (By.XPATH, "//div[@title='Proceed']")
        wait.until(EC.element_to_be_clickable(proceed_button)).click()
        try:
            error_locator = (By.XPATH, "//div[contains(@class, 'MuiFormHelperText-root') and contains(@class, 'Mui-error')]")
            error_element = wait.until(EC.visibility_of_element_located(error_locator))
            error_text = error_element.text
            assert "Please enter email" in error_text, f"Empty email validation failed! Message was: '{error_text}'"
            logging.info("Empty email validation passed.")
        except TimeoutException:
            logging.error("Empty email validation failed. Error message not found!")

if __name__ == "__main__":
    try:
        BrowserHandler.initialize_browser()
        ResetPasswordTests.navigate_to_reset()
        ResetPasswordTests.test_empty_email()
    except Exception as e:
        logging.error(f"Test execution failed: {e}")
    finally:
        BrowserHandler.shutdown_browser()


INFO:root:Testing empty email validation...
ERROR:root:Empty email validation failed. Error message not found!
